<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-18 15:22:45
-------------------
qualified stocks: 94
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  1.01 L
Current:  1.36 C
-------------------
Today PnL: 1.94 L (1.44%)
Current PnL: -18.82 L (-12.8%)
CY Booked + Current PnL: -10.55 L (-7.17%)
-------------------
Total profit:  3.35 L
Total loss:  -22.17 L
-------------------
Total Booked + Current PnL: 16.43 L (13.69%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.08%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 78.44 L (57.65%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.02%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-17.42,H-LC,6.56,114248.0,9936.0,6718.0,2.70,9.53,5.88,15.96,9.0,1.48,0.85,28.03,X5K,ATH,METALS
52,LALPATHLAB,2838.53,3545.00,-21.79,H-SC,10.95,233696.0,40676.0,7361.0,2.66,21.07,3.15,24.89,137.0,5.53,1.73,49.34,X40N,NTT,HEALTHCARE
69,SBIN,760.30,863.00,-11.34,M-LC,3.85,219155.0,17676.0,9533.0,0.05,8.77,4.35,13.51,60.0,1.85,1.62,22.47,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-13.75,H-LC,6.24,172249.0,16072.0,10025.0,0.10,10.29,5.82,16.71,22.0,1.60,1.28,29.36,X40N,BTT,HEALTHCARE
47,ITC,411.90,452.00,-37.50,H-LC,2.85,149768.0,-164.0,14767.0,-0.64,-0.11,9.86,9.74,4.0,-0.01,1.11,6.12,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RAJOOENG,102.50,143.33,-32.80,H-SC,6.05,101920.0,-580.0,41410.0,-0.62,-0.57,40.63,39.83,114.0,-0.01,0.75,0.91,AR,ATH,MISC
59,PGHH,13388.00,18062.58,-30.68,H-MC,6.27,200580.0,-240.0,70363.0,0.78,-0.12,35.08,34.92,80.0,-0.00,1.49,4.55,X40,ATH,FMCG
57,NATIONALUM,189.63,247.44,-50.29,H-MC,2.00,99725.0,-779.0,31423.0,0.31,-0.77,31.51,30.49,79.0,-0.02,0.74,31.38,MH,ATH,METALS
93,WIPRO,243.46,420.00,-13.34,M-LC,7.64,152005.0,1060.0,108395.0,-0.66,0.70,71.31,72.51,53.0,0.01,1.13,6.77,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.44,H-LC,2.81,214569.0,10069.0,31349.0,0.14,4.92,14.61,20.25,10.0,0.32,1.59,14.09,X40N,BTT,PHARMA
47,ITC,411.90,452.0,-37.50,H-LC,2.85,149768.0,-164.0,14767.0,-0.64,-0.11,9.86,9.74,4.0,-0.01,1.11,6.12,X40,NTT,FMCG
58,NESTLEIND,1134.30,1377.0,-13.06,H-LC,3.41,267602.0,2176.0,54618.0,4.98,0.82,20.41,21.40,11.0,0.04,1.98,8.21,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-9.79,H-LC,3.73,175902.0,-25484.0,113105.0,2.30,-12.65,64.30,43.51,27.0,-0.23,1.30,22.45,X40,BTT,PAINTS
54,LTIM,5564.16,7230.2,-7.51,H-LC,3.73,183672.0,-16638.0,76610.0,-0.12,-8.31,41.71,29.94,16.0,-0.22,1.36,26.89,X200,ATH,IT


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,RELAXO,902.64,1176.00,-35.42,H-SC,10.67,59910.0,-55628.0,90620.0,7.20,-48.15,151.26,30.28,136.0,-0.61,0.44,17.04,X40N,NTT,FOOTWEAR
15,BLUESTARCO,1646.70,2326.38,10.70,H-MC,6.58,191030.0,26360.0,41606.0,7.01,16.01,21.78,41.28,89.0,0.63,1.41,25.27,X40N,ATH,AC
13,BATAINDIA,1550.24,2096.00,-37.72,M-SC,7.47,93084.0,-35586.0,80881.0,6.84,-27.66,86.89,35.20,219.0,-0.44,0.69,6.84,X40,NTT,FOOTWEAR
35,HEROMOTOCO,4311.81,5949.07,-5.85,H-MC,8.57,174293.0,23380.0,33917.0,5.77,15.49,19.46,37.97,93.0,0.69,1.29,44.72,AR,ATH,AUTO
91,VOLTAS,1278.28,1918.49,-3.60,H-MC,7.69,202125.0,10383.0,85640.0,5.70,5.42,42.37,50.08,99.0,0.12,1.50,12.58,XY25,ATH,AC


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,MEDANTA,1087.93,1486.00,9.08,H-MC,13.84,155718.0,29518.0,16662.0,-2.31,23.39,10.70,36.59,108.0,1.77,1.15,38.84,XY24,NTT,HEALTHCARE
30,GREENPANEL,375.16,537.00,192.64,M-SC,5.43,130784.0,-52294.0,131268.0,-1.89,-28.56,100.37,43.14,230.0,-0.40,0.97,23.02,XY24,NTT,MISC
72,SIEMENS,4406.90,7969.85,-3.04,H-LC,35.02,77875.0,-32297.0,121368.0,-1.70,-29.32,155.85,80.85,15.0,-0.27,0.58,14.70,AR,ATH,ELECTRICAL
74,SONACOMS,578.05,1006.00,-37.48,M-SC,13.62,77875.0,-23284.0,98177.0,-1.52,-23.02,126.07,74.03,202.0,-0.24,0.58,9.81,AR,BTT,AUTO
45,INFY,1461.46,2275.00,-21.39,H-LC,11.27,256972.0,-4629.0,150252.0,-0.84,-1.77,58.47,55.67,3.0,-0.03,1.90,4.23,X40,BTT,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TTKPRESTIG,769.29,770.0,95.94,M-SC,10.13,84430.0,-16347.0,16439.0,2.03,-16.22,19.47,0.09,245.0,-0.99,0.63,10.92,OX40N,NTT,DURABLES
85,UJJIVANSFB,52.77,60.0,84.36,H-SC,20.15,116235.0,-26244.0,45762.0,3.31,-18.42,39.37,13.70,163.0,-0.57,0.86,37.19,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-17.90,H-SC,3.14,117000.0,-26803.0,53551.0,1.59,-18.64,45.77,18.60,149.0,-0.50,0.87,29.11,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,115.40,M-SC,2.56,141372.0,-33187.0,33222.0,2.97,-19.01,23.50,0.02,221.0,-1.00,1.05,22.59,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.07,H-SC,3.05,213291.0,-56376.0,92718.0,0.65,-20.91,43.47,13.47,138.0,-0.61,1.58,7.88,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TCS,3794.03,4998.00,-29.01,H-LC,16.10,235053.0,-60881.0,154782.0,-0.29,-20.57,65.85,31.73,1.0,-0.39,1.74,0.35,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.39,H-LC,11.27,256972.0,-4629.0,150252.0,-0.84,-1.77,58.47,55.67,3.0,-0.03,1.90,4.23,X40,BTT,IT
47,ITC,411.90,452.00,-37.50,H-LC,2.85,149768.0,-164.0,14767.0,-0.64,-0.11,9.86,9.74,4.0,-0.01,1.11,6.12,X40,NTT,FMCG
90,VBL,492.64,671.64,-11.61,H-LC,10.05,268462.0,3422.0,92888.0,-0.57,1.29,34.60,36.33,5.0,0.04,1.99,14.76,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.10,H-LC,10.81,196580.0,-19939.0,112857.0,0.21,-9.21,57.41,42.91,7.0,-0.18,1.46,2.95,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.0,-21.79,H-SC,10.95,233696.0,40676.0,7361.0,2.66,21.07,3.15,24.89,137.0,5.53,1.73,49.34,X40N,NTT,HEALTHCARE
56,MEDANTA,1087.93,1486.0,9.08,H-MC,13.84,155718.0,29518.0,16662.0,-2.31,23.39,10.70,36.59,108.0,1.77,1.15,38.84,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-10.62,H-LC,6.67,223195.0,38714.0,31649.0,4.52,20.99,14.18,38.14,38.0,1.22,1.65,33.82,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,1646.70,2326.38,10.70,H-MC,6.58,191030.0,26360.0,41606.0,7.01,16.01,21.78,41.28,89.0,0.63,1.41,25.27,X40N,ATH,AC
56,MEDANTA,1087.93,1486.00,9.08,H-MC,13.84,155718.0,29518.0,16662.0,-2.31,23.39,10.70,36.59,108.0,1.77,1.15,38.84,XY24,NTT,HEALTHCARE
51,KPIGREEN,497.21,731.35,9.02,H-SC,11.62,129944.0,4647.0,54356.0,1.38,3.71,41.83,47.09,141.0,0.09,0.96,62.63,MH,ATH,POWER
91,VOLTAS,1278.28,1918.49,-3.60,H-MC,7.69,202125.0,10383.0,85640.0,5.70,5.42,42.37,50.08,99.0,0.12,1.50,12.58,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-4.05,H-SC,4.19,109175.0,7171.0,36759.0,0.79,7.03,33.67,43.06,122.0,0.20,0.81,25.35,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-5.85,H-MC,8.57,174293.0,23380.0,33917.0,5.77,15.49,19.46,37.97,93.0,0.69,1.29,44.72,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.09,H-SC,11.77,136337.0,13001.0,120767.0,1.04,10.54,88.58,108.45,119.0,0.11,1.01,35.25,AR,ATH,MISC
51,KPIGREEN,497.21,731.35,9.02,H-SC,11.62,129944.0,4647.0,54356.0,1.38,3.71,41.83,47.09,141.0,0.09,0.96,62.63,MH,ATH,POWER
92,WHIRLPOOL,1219.98,2270.00,-42.31,M-SC,4.52,97500.0,6002.0,72754.0,4.78,6.56,74.62,86.07,223.0,0.08,0.72,41.04,XR,NTT,DURABLES
93,WIPRO,243.46,420.00,-13.34,M-LC,7.64,152005.0,1060.0,108395.0,-0.66,0.70,71.31,72.51,53.0,0.01,1.13,6.77,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.32,72710.0,-40839.0,80890.0,-0.24,-35.97,111.25,35.27,268.0,-0.50,0.54,77.96,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.50,H-SC,15.38,89750.0,-11213.0,105743.0,0.67,-11.11,117.82,93.63,148.0,-0.11,0.66,33.68,SR,ATH,CHEMICALS
55,MASFIN,326.60,399.50,-19.15,H-SC,13.20,92010.0,-5970.0,27842.0,2.06,-6.09,30.26,22.32,152.0,-0.21,0.68,33.45,XR,ATH,FINANCE
89,VALIANTORG,512.64,838.00,-327.07,H-SC,16.44,96642.0,-36644.0,121237.0,0.85,-27.49,125.45,63.47,139.0,-0.30,0.72,62.00,XR,NTT,CHEMICALS
36,HINDALCO,651.95,756.01,-17.42,H-LC,6.56,114248.0,9936.0,6718.0,2.70,9.53,5.88,15.96,9.0,1.48,0.85,28.03,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-5.85,H-MC,8.57,174293.0,23380.0,33917.0,5.77,15.49,19.46,37.97,93.0,0.69,1.29,44.72,AR,ATH,AUTO
52,LALPATHLAB,2838.53,3545.00,-21.79,H-SC,10.95,233696.0,40676.0,7361.0,2.66,21.07,3.15,24.89,137.0,5.53,1.73,49.34,X40N,NTT,HEALTHCARE
28,DMART,3764.92,5201.00,-10.62,H-LC,6.67,223195.0,38714.0,31649.0,4.52,20.99,14.18,38.14,38.0,1.22,1.65,33.82,X40N,NTT,FMCG
5,ASIANPAINT,2961.56,4250.00,-9.79,H-LC,3.73,175902.0,-25484.0,113105.0,2.30,-12.65,64.30,43.51,27.0,-0.23,1.30,22.45,X40,BTT,PAINTS
4,APOLLOHOSP,7098.95,8285.00,-13.75,H-LC,6.24,172249.0,16072.0,10025.0,0.10,10.29,5.82,16.71,22.0,1.60,1.28,29.36,X40N,BTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.37
1,25,40.88
2,50,70.50


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.54
LC    35.48
MC    25.98
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.08
X40N     15.40
X40      14.20
XY25     11.48
XR        8.86
AR        8.23
OX40N     7.37
X5K       2.23
MH        1.70
X200      1.36
SR        1.09
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.33
H-SC    24.02
H-MC    23.09
M-SC    13.08
M-LC     7.09
M-MC     2.56
L-SC     1.44
L-LC     1.06
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.04,-2.07,33.33
IT,11.61,-18.85,78.21
FINANCE,8.02,-20.37,64.90
BANKS,7.45,-13.23,61.31
MISC,5.72,-18.90,87.06
PAINTS,5.53,-16.92,39.97
HEALTHCARE,5.19,6.38,17.26
ELECTRICAL,4.67,-11.63,59.98
AUTO,4.46,-12.40,61.29


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2844987.0,23
AR,1096501.0,10
XR,1096410.0,13
X40,819974.0,11
X40N,527083.0,12
XY25,498203.0,8
OX40N,464729.0,10
SR,195006.0,2
X5K,138398.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2770423.0,28
M-SC,1479155.0,17
H-MC,1259654.0,18
H-LC,1212799.0,19
M-LC,421907.0,5
M-MC,332160.0,2
L-SC,266676.0,3
L-MC,57668.0,1
L-LC,43238.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      937805.0      6
M-SC       XY24      867914.0      7
H-SC       AR        666808.0      5
H-LC       X40       529830.0      6
H-SC       XR        516152.0      6
H-MC       XY24      494601.0      5
M-MC       XY24      332160.0      2
H-LC       AR        234225.0      2
           X40N      210113.0      5
H-MC       X40       209263.0      4
           XY25      205958.0      2
H-SC       OX40N     203434.0      4
           X40N      196862.0      4
           SR        195006.0      2
L-SC       XR        176655.0      2
M-LC       XY24      172299.0      2
M-SC       AR        161551.0      2
M-LC       X5K       131680.0      1
M-SC       XY25      124379.0      1
           OX40N     122498.0      4
           XR        121932.0      2
H-MC       X40N      120108.0      3
           XR        115608.0      1
H-LC       XY25      115095.0      3
M-LC       XR        108395.0      1
L-SC       OX40N      90021.0      1
M-SC       X40        80881.0      1
H-LC       X200       76610.0      1
L-MC       XR         57668.0      1
H-SC       MH         54356.0      1
H-MC       OX40N      48776.0      1
L-LC       XY25       43238.0      1
H-LC       XY24       40208.0      1
H-MC       AR         33917.0      1
           MH         31423.0      1
M-LC       XY25        9533.0      1
H-LC       X5K         6718.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
